In [ ]:
# Copyright (c) 2024, InfinityQ Technology Inc.

import numpy as np
import utils
import problem_gen

from titanq import Model, Vtype, Target, S3Storage
import logging

### Setting Credentials for TitanQ
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

In [ ]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API Key Here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Your Access key"
# AWS_SECRET_ACCESS_KEY = "Your secret access key"
# AWS_BUCKET_NAME = "Your bucket name"

### Setting Up Data

In [ ]:
jobs = utils.read_instance("instances/la01")

# Number of jobs
num_jobs = len(jobs)
# Number of machines
num_machines = utils.get_num_machines(jobs)
# Number of tasks
num_tasks = utils.get_num_tasks(jobs)

machine_names = [f"Machine_{i}" for i in range(num_machines)]
task_names = []
assignment = {}
for job, tasks in jobs.items():
    task_names.extend(
        [f"Job{job+1}_Task{task_id+1}" for task_id in range(len(tasks))])
    assignment.update(
        {f"Job{job+1}_Task{task_id+1}": machine_names[task[0]] for task_id, task in enumerate(tasks)})

# Available Time Slots
## Each row represents a machine, and each column represents an interval of time.
## An entry of 1 indicates that the machine is available, and an entry of -1 indicates that the machine is not available.

## Set the size of the available_time_slots matrix
rows = num_machines
columns = 500

## Create a matrix with all elements initialized to 1
available_time_slots = np.ones((rows, columns), dtype=int)

# Blocked Time slots
## Ex: 'Machine_0' is not available during time unit 1 to 4
## available_time_slots[0, 1:4] = -1

# Distance matrix for moving from one machine to another
distance_matrix = np.zeros((num_machines, num_machines), dtype=np.float32)

### Building the JSSP Model

In [ ]:
# Model generation
tasks = [item for sublist in jobs.values()
        for item in sublist] # List of tasks

max_start_time = utils.get_max_time(jobs)*2

Nx = num_tasks
Nz = len([(i, j) for i in range(Nx) for j in range(Nx) if tasks[i][0] == tasks[j][0]])

# Variable array structure
# x = [x_0, x_1, ..., x_num_tasks, z_01, z_02, z_03, z_12, z_13, z_23,
#   y_{machine_1}l_1_{task_1},y_{machine_1}l_2_{task_1},...,y_{machine_1}l_{num_blocked}_{task_1},....,y_{machine_m}l_j_{task_1}]
N = Nx + Nz

machine_group = utils.generate_machine_group(jobs)

Ny = sum([len(utils.find_available_time_slots(available_time_slots[m_idx]))*len(machine_group[m_idx])
         for m_idx in range(num_machines) if m_idx in machine_group.keys()])
N += Ny

# Large value to enforce tasks not overlapping with each other
V = sum(task[1] for task in tasks)

# Large number to enforce tasks not overlapping with blocked time slots
H = V

constraint_weights, constraint_bounds = problem_gen.generate_constraint_weights_bounds(
    jobs,
    N,
    Nx,
    Nz,
    V,
    distance_matrix,
    num_machines,
    available_time_slots,
    H
)

W, b = problem_gen.generate_weights_bias(jobs, N)

variable_bounds = problem_gen.generate_variable_bounds(
    jobs,
    distance_matrix,
    Nx,
    Nz,
    Ny,
    max_start_time
)
variable_types = problem_gen.generate_variable_types(Ny, Nz, Ny)

num_variables = N

### Building the Model on TitanQ

In [ ]:
############
# TitanQ SDK
############
model_JSSP = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)

model_JSSP.add_variable_vector(
    'x', Nx, Vtype.INTEGER, variable_bounds=variable_bounds[:Nx].tolist())
model_JSSP.add_variable_vector('z', Nz, Vtype.BINARY)
model_JSSP.add_variable_vector('y', Ny, Vtype.BINARY)
model_JSSP.set_objective_matrices(W, b, Target.MINIMIZE)
model_JSSP.add_inequality_constraints_matrix(
    constraint_weights, constraint_bounds)

### Setting TitanQ Hyperparameters

In [ ]:
num_chains = 64
num_engines = 1
T_min = 0.1
T_max = 1e6
beta = (1.0/np.geomspace(T_min, T_max, num_chains)).tolist()
timeout_in_seconds = 100.0

### Sending the Model to TitanQ Solver

In [ ]:
response = model_JSSP.optimize(
    beta=beta,
    timeout_in_secs=timeout_in_seconds,
    num_engines=num_engines,
    num_chains=num_chains
)

print("-" * 15, "+", "-" * 26, sep="")
print("Ising energy   | Result vector")
print("-" * 15, "+", "-" * 26, sep="")
ctr = 0
for ising_energy, result_vector in response.result_items():
    print(f"{ising_energy: <14f} | {result_vector}")
    if ctr == 0:
        lowest_ising_energy = ising_energy
        index = 0
    elif ising_energy < lowest_ising_energy:
        lowest_ising_energy = ising_energy
        index = ctr
    ctr += 1

### GANTT Chart of Raw Schedule

In [ ]:
# The schedule dictionary
ground_state = response.result_vector()[index]
schedule = utils.extract_solution(ground_state[:Nx], tasks, task_names)

# Post-processing for adding the hand-offs
for i, task in enumerate(task_names):
    if i == len(task_names) - 2:
        break
    schedule[f"Hand-off: {assignment[task_names[i]]}-{task_names[i]} --> {assignment[task_names[i+1]]}-{task_names[i+1]}"] = \
        (schedule[task_names[i]][1], schedule[task_names[i]][1] + int(distance_matrix[machine_names.index(
            assignment[task_names[i]])][machine_names.index(assignment[task_names[i+1]])]))
    assignment[f"Hand-off: {assignment[task_names[i]]}-{task_names[i]} --> {assignment[task_names[i+1]]}-{task_names[i+1]}"] = assignment[task_names[i]]

utils.plot_schedule(
    assignment,
    schedule,
    available_time_slots,
    machine_names,
    unit="days"
)

### Extracting the Schedule Finish Time

In [ ]:
utils.max_value_schedule(schedule)